In [8]:
import numpy as np
import cvxpy as cp
from public.basis_generator_v2 import rand_moment
from numpy import linalg as LA

In [18]:
# Parameters
dim = 2
num_obs = 3
len_seq = 2
num_out = 2
remove_last_out = True
level = 1

flag = False
count = 1
stop = 10000
norm_sens = 0.000000001

X_basis = []

# Initialization
X = rand_moment(dim,
             num_obs,
             len_seq,
             num_out,
             sel_sequences = [len_seq],
             remove_last_out = True)

X = X/LA.norm(X)
X_basis.append(X)

while flag==False:
    X = rand_moment(dim,
             num_obs,
             len_seq,
             num_out,
             sel_sequences = [len_seq],
             remove_last_out = True)
    
    for k in range(len(X_basis)):
        X -= X_basis[k]*np.sum(X_basis[k]*np.conjugate(X))
    if LA.norm(X) < norm_sens:
        print("Nul matrix found")
        print("Number of LI moment matrices: ", len(X_basis))
        flag=True
    else:
        X = X/LA.norm(X)
        X_basis.append(X)
        count+=1
    if count > stop:
        print("Cannot find the basis")
        
        
X_basis_2d = X_basis

Nul matrix found
Number of LI moment matrices:  35


In [19]:
X_basis_2d[0].shape

(10, 10)

In [59]:
# Parameters
dim = 3
num_obs = 3
len_seq = 2
num_out = 2
remove_last_out = True
level = 1

X = rand_moment(dim,
             num_obs,
             len_seq,
             num_out,
             sel_sequences = [len_seq],
             remove_last_out = True)

In [69]:
Eta = []

for k in range(100):
    X = rand_moment(dim,
             num_obs,
             len_seq,
             num_out,
             sel_sequences = [len_seq],
             remove_last_out = True)
    
    eta = cp.Variable((1, 1))
    alpha = cp.Variable((len(X_basis_2d), 1))
    beta = cp.Variable((len(X_basis_2d), 1))
    M = cp.Variable(X_basis_2d[0].shape)
    N = cp.Variable(X_basis_2d[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis_2d[j] for j in range(len(X_basis_2d))])]
    constraints += [N[0,0] == 1 - eta]

    for i in range(1,len(X)):
        constraints += [
            eta*X[i,i] + N[i,i] == M[i,i]
        ]

    constraints += [M >> 0]
    constraints += [M == sum([alpha[j]*X_basis_2d[j] for j in range(len(X_basis_2d))])]
    constraints += [M[0,0] == 1]

    prob = cp.Problem(cp.Maximize(eta),
                      constraints)

    prob.solve(solver=cp.MOSEK, verbose=False)
    Eta.append(eta.value[0][0])

In [70]:
min(Eta)

0.9514229648993894

In [71]:
max(Eta)

1.0000000000007403